# Identifying Food Deserts
### Independent project, serving as capstone to the IBM Coursera Data Science Professional Certificate
#### Christian Capdeville, November 2020

#### Overview

Many low income areas across the US are without easy access to food of adequate nutritional quality (for example, a full-size grocery store such as Kroger's or HEB). Instead, these places may have nearby convenience stores or fast food restaurants, which exacerbate related health issues which often plague low income communities. An area which can be described thusly is referred to as a "food desert." Most residents of low income food deserts also lack adequate transportation, presenting another barrier to overcoming the situation. 

Non-profit organizations looking to improve lives within low-income communities often look to resolve this situation. Some organizations open up their own grocery store, while others might look to solve for the transportation barrier. In this project, I will be looking to identify food deserts to highlight for non-profit organizations looking to deploy resources.

Some of the criteria for this identification will be:
- Mean distance to grocery stores
- Mean distance to convenience stores
- Mean distance to fast food restaurants
- Population or density of residential addresses/apartment complexes

#### Data needed

I will use Foursquare location data to identify neighborhoods with high distance to grocery stores, low distance to convenience stores and fast food, and potentially high density of residential apartment complexes. I will use an area of Galveston known as "North of Broadway" as a model, as this is a food desert which is known to me. I will look for other areas in Houston which match this dynamic.

I will also look to incorporate specific guidelines of distance and community constraints which are commonly cited:
- More than one mile from a grocery store
- Transportation constraints